<a href="https://colab.research.google.com/github/Gurukannan/Billing-Application/blob/master/Cosine_Similarity_with_Glove.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import nltk
import spacy
import os
import string
import gensim

In [ ]:
train = pd.read_excel("train2.xlsx")
train = train.dropna()
train

,id,qid1,qid2,question1,question2,is_duplicate,Q1,Q2
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,"['step', 'step', 'guide', 'invest', 'share']","['step', 'step', 'guide', 'invest', 'share']"
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,"['story', 'kohinoor', 'kohinoor', 'diamond']","['would', 'happen', 'indian', 'government']"
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,"['increase', 'speed', 'internet', 'connection']","['internet', 'speed', 'increased', 'hacking']"
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,"['mentally', 'lonely', 'solve']","['find', 'remainder', 'math2324math', 'divided']"
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,"['one', 'dissolve', 'water', 'quikly', 'sugar']","['fish', 'would', 'survive', 'salt', 'water']"


In [ ]:
!wget -c http://nlp.stanford.edu/data/glove.840B.300d.zip

--2021-03-25 11:57:34--  http://nlp.stanford.edu/data/glove.840B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.840B.300d.zip [following]
--2021-03-25 11:57:34--  https://nlp.stanford.edu/data/glove.840B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip [following]
--2021-03-25 11:57:34--  http://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 206 Partial Content
Length: 2176768927 (2.0G), 21

In [ ]:
import zipfile
with zipfile.ZipFile("/content/glove.840B.300d.zip", "r") as zipread:
  zipread.extractall("/content/")
  zipread.close

In [ ]:
import gensim
from gensim.models import Word2Vec
from gensim.scripts.glove2word2vec import glove2word2vec

PATH_TO_GLOVE = "/content/glove.840B.300d.txt"

tmp_file = "/glove.840B.300d.w2v.txt"
glove2word2vec(PATH_TO_GLOVE, tmp_file)
glove = gensim.models.KeyedVectors.load_word2vec_format(tmp_file)

In [ ]:
import scipy
from ast import literal_eval
def cosine_distance_wordembedding_method(s1, s2):
    vector_1 = np.mean([glove[word] for word in literal_eval(s1) if word in glove],axis=0)
    vector_2 = np.mean([glove[word] for word in literal_eval(s2) if word in glove],axis=0)
    cosine = scipy.spatial.distance.cosine(vector_1, vector_2)
    return 1-cosine

In [ ]:
train['Q2']

0        ['step', 'step', 'guide', 'invest', 'share']
1         ['would', 'happen', 'indian', 'government']
2       ['internet', 'speed', 'increased', 'hacking']
3    ['find', 'remainder', 'math2324math', 'divided']
4       ['fish', 'would', 'survive', 'salt', 'water']
Name: Q2, dtype: object

In [ ]:
train["cos_sim"] = train.apply(lambda x: cosine_distance_wordembedding_method(x.Q1, x.Q2), axis=1)
train.head()  

,id,qid1,qid2,question1,question2,is_duplicate,Q1,Q2,cos_sim
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,"['step', 'step', 'guide', 'invest', 'share']","['step', 'step', 'guide', 'invest', 'share']",1.000000
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,"['story', 'kohinoor', 'kohinoor', 'diamond']","['would', 'happen', 'indian', 'government']",0.134267
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,"['increase', 'speed', 'internet', 'connection']","['internet', 'speed', 'increased', 'hacking']",0.907435
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,"['mentally', 'lonely', 'solve']","['find', 'remainder', 'math2324math', 'divided']",0.495659
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,"['one', 'dissolve', 'water', 'quikly', 'sugar']","['fish', 'would', 'survive', 'salt', 'water']",0.763828


In [ ]:
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(train[["cos_sim"]], train["is_duplicate"], random_state=5, test_size=0.2)

In [ ]:
import xgboost as xgb
options = {"model_param":
         {"eta": 0.15, 
         "max_depth": 4,
         "n_estimators":50,
         "sub_sample":0.5,
         "scale_pos_weight":1.75,
         "random_state":1,
         "eval_metric": "logloss",
         "objective": "binary:logistic"},
         "num_round":50,
         "test_frac":0.2}

In [ ]:
#training model
xgbmatrix = xgb.DMatrix(train_X, train_y)
xgb_model = xgb.train(options["model_param"], xgbmatrix, options["num_round"], verbose_eval=10)
xgb_valmatrix = xgb.DMatrix(val_X)
y_val_pred = xgb_model.predict(xgb_valmatrix)
y_val_pred #just checking

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


array([0.39110422, 0.17844072, 0.65599036, ..., 0.5713001 , 0.3393073 ,
       0.7067311 ], dtype=float32)

In [ ]:
y_val_pred_binary = [1 if y>0.5 else 0 for y in y_val_pred]
from sklearn.metrics import confusion_matrix, classification_report

confusion_matrix(val_y, y_val_pred_binary)

array([[29799, 21346],
       [ 6083, 23630]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(val_y, y_val_pred_binary)

0.6607756808231715